Inprocessing refers to mitigating the bias in model's predictions during training.
In-processing techniques have several advantages over pre-processing techniques. They can be more effective at addressing bias that is deeply embedded in the data, and they do not require any modifications to the data collection or preprocessing pipeline

1.  Prejudice Remover : 
Prejudice remover works by adding a regularization term to the model's loss function. This regularization term penalizes the model for making predictions that are inconsistent with the desired fairness criteria.

Given the fact that our baseline model was baised against specific gender and race, using prejudice remover should help us get better fairness

In [3]:
import aif360
import holisticai
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from holisticai.bias.mitigation import PrejudiceRemover

Data Gathering

In [4]:
baseline_preprocessed_df = pd.read_csv("C:/Users/athar/OneDrive/Desktop/Rutgers/Ethical Stat Learning/Project/data/data_preprocessed_baseline.csv",index_col=0)
baseline_preprocessed_df = baseline_preprocessed_df.drop(['id','age_cat'],axis=1)

In [5]:
protected_variables = ["sex", "African-American_race"]
output_variable = ["two_year_recid"]

In [6]:
# Simple preprocessing
y = baseline_preprocessed_df[output_variable]
X = pd.get_dummies(baseline_preprocessed_df.drop(output_variable, axis=1))
group = ["African-American_race"]
group_a = baseline_preprocessed_df["African-American_race"] == 1
group_b = baseline_preprocessed_df["African-American_race"] == 0
data_ = [X, y, group_a, group_b]


In [7]:
# Train test split
dataset = train_test_split(*data_, test_size=0.2, shuffle=True)
train_data = dataset[::2]
test_data = dataset[1::2]

In [8]:
# efficacy metrics from sklearn
from sklearn import metrics

# dictionnary of metrics
metrics_dict={
        "Accuracy": metrics.accuracy_score,
        "Balanced accuracy": metrics.balanced_accuracy_score,
        "Precision": metrics.precision_score,
        "Recall": metrics.recall_score,
        "F1-Score": metrics.f1_score}

# efficacy metrics dataframe helper tool
def metrics_dataframe(y_pred, y_true, metrics_dict=metrics_dict):
    metric_list = [[pf, fn(y_true, y_pred)] for pf, fn in metrics_dict.items()]
    return pd.DataFrame(metric_list, columns=["Metric", "Value"]).set_index("Metric")

In [9]:
# import
from holisticai.bias.mitigation import GridSearchReduction

In [89]:
# data and model
X_train, y_train, group_a_train, group_b_train = train_data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
model_lr = LogisticRegression()
model_lr.fit(X_train_scaled,y_train)

c:\Users\athar\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [90]:
# initialize
gsr = GridSearchReduction(constraints="FalsePositiveRateParity")

# incorporate model in gsr
gsr.transform_estimator(model_lr)

GridSearchReduction(constraints='FalsePositiveRateParity')

In [91]:
# fit with data
gsr.fit(X_train_scaled, y_train, group_a_train, group_b_train)

GridSearchReduction(constraints='FalsePositiveRateParity')

In [92]:
# predict test
X_test, y_test, group_a_test, group_b_test = test_data
X_test_scaled = scaler.transform(X_test)
y_pred = gsr.predict(X_test_scaled)
y_proba = gsr.predict_proba(X_test_scaled)
y_score = y_proba[:,1]
y_true = y_test

In [93]:
# Baseline efficacy
metrics_dataframe(y_pred, y_true)

,Value
Metric,
Accuracy,0.704941
Balanced accuracy,0.694596
Precision,0.714286
Recall,0.582822
F1-Score,0.641892


In [94]:
# bias
# holisticai imports
from holisticai.bias.metrics import classification_bias_metrics
classification_bias_metrics(group_a_test, group_b_test, y_pred, y_true, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,0.178421,0
Disparate Impact,1.641028,1
Four Fifths Rule,0.609374,1
Cohen D,0.375973,0
2SD Rule,7.000460,0
Equality of Opportunity Difference,0.116075,0
False Positive Rate Difference,0.142144,0
Average Odds Difference,0.129110,0
Accuracy Difference,-0.054764,0


<b>prejudice remover

In [65]:
model_lr2 = LogisticRegression()
model_lr2.fit(X=X_train_scaled,y=y_train)
print(model_lr2.n_features_in_)
print(model_lr2.coef_)
print(model_lr2.intercept_)


[[-0.38195791  0.04428885  0.04869928  0.03057386  0.54646495  0.68172774
   0.08529593  0.10712253 -0.02976715  0.02975681 -0.02390729  0.01100644
  -0.00288405 -0.07262317  0.07262317  0.11138892 -0.12070439  0.03608336
   0.1100735  -0.04757364 -0.0248501 ]]
[-0.22378369]
21


c:\Users\athar\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [102]:
pr2.transform_estimator?

Signature: pr2.transform_estimator()
Docstring: <no docstring>
File:      c:\users\athar\anaconda3\lib\site-packages\holisticai\bias\mitigation\inprocessing\prejudice_remover\transformer.py
Type:      method


In [96]:
pr2 = PrejudiceRemover(eta=1,C=0.5,init_type='StandarLR')
pr2.transform_estimator()

PrejudiceRemover(C=0.5, eta=1, init_type='StandarLR')

In [97]:
pr2.fit( X=X_train_scaled, y_true=y_train, group_a=group_a_train, group_b=group_b_train)

PrejudiceRemover(C=0.5, eta=1, init_type='StandarLR')

In [98]:
pr2.get_params(deep=True)

{'C': 0.5,
 'eta': 1,
 'fit_intercept': True,
 'init_type': 'StandarLR',
 'maxiter': 1000,
 'penalty': 'l2',
 'print_interval': 20,
 'verbose': 0}

In [99]:
# predict test
y_pred = pr2.predict(X_test_scaled,group_a=group_a_test,group_b=group_b_test)
y_proba = pr2.predict_proba(X_test_scaled,group_a=group_a_test,group_b=group_b_test)
y_score = y_proba[:,1]
y_true = y_test

In [100]:
# Baseline efficacy
metrics_dataframe(y_pred, y_true)

,Value
Metric,
Accuracy,0.704941
Balanced accuracy,0.693167
Precision,0.723529
Recall,0.565951
F1-Score,0.635112


In [101]:
# bias
# holisticai imports
from holisticai.bias.metrics import classification_bias_metrics
classification_bias_metrics(group_a_test, group_b_test, y_pred, y_true, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,0.157050,0
Disparate Impact,1.573107,1
Four Fifths Rule,0.635685,1
Cohen D,0.332729,0
2SD Rule,6.218309,0
Equality of Opportunity Difference,0.118887,0
False Positive Rate Difference,0.101147,0
Average Odds Difference,0.110017,0
Accuracy Difference,-0.035261,0
